<a href="https://colab.research.google.com/github/FatiZaha/Machine-Learning/blob/main/HerbsGuide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords

# Télécharger les stop words en français
nltk.download('stopwords')
stop_words_fr = stopwords.words('french')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [37]:
# Charger les données depuis le fichier CSV
df = pd.read_csv('plantes_medicinales.csv')

# Appliquer une fonction pour tronquer les valeurs à 10 caractères et ajouter "..."
df_tronque = df.map(lambda x: str(x)[:10] + "..." if len(str(x)) > 10 else str(x))

# Afficher les 5 premières lignes pour vérifier
print(df_tronque.head(5))


          Plante      Bienfaits   Utilisations    Précautions
0  Menthe poi...  Soulage le...  Infusion, ...  Éviter en ...
1      Camomille  Réduit le ...  Infusion, ...  Pas recomm...
2      Aloe Vera  Apaise les...       Gel, jus  Ne pas app...
3      Gingembre  Aide à la ...  Infusion, ...  Peut inter...
4           Thym  Stimule le...  Infusion, ...  Utilisatio...


In [38]:
# Fonction pour nettoyer et vectoriser les descriptions des plantes
def vectoriser_texte(texte_plantes):
    tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words_fr)  # Enlever les mots fréquents en français
    tfidf_matrix = tfidf_vectorizer.fit_transform(texte_plantes)  # Convertir le texte en une matrice TF-IDF
    return tfidf_matrix

In [41]:
# Fonction de recommandation basée sur les symptômes de l'utilisateur
def recommander_plantes(symptome_utilisateur, df):
    # Combiner les colonnes de 'Bienfaits' et 'Utilisations' pour former un seul texte pour chaque plante
    textes_plantes = df['Bienfaits'] + " " + df['Utilisations']

    # Ajouter le symptôme de l'utilisateur à la liste des plantes
    textes_plantes = list(textes_plantes) + [symptome_utilisateur]

    # Vectoriser les textes (plantes + symptôme utilisateur)
    tfidf_matrix = vectoriser_texte(textes_plantes)

    # Calculer la similarité cosinus entre le symptôme utilisateur et les plantes
    similarity_matrix = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

    # Créer une liste des plantes avec leur score de similarité
    scores_similarite = similarity_matrix.flatten()

    # Ajouter les scores de similarité aux données des plantes
    df['score_similarite'] = scores_similarite

    # Trier les plantes par score de similarité (de la plus pertinente à la moins pertinente)
    recommandations = df[['Plante', 'Bienfaits', 'Utilisations', 'Précautions', 'score_similarite']].sort_values(by='score_similarite', ascending=False)

    return recommandations[['Plante', 'Bienfaits', 'Utilisations', 'Précautions', 'score_similarite']].head(5)  # Limiter aux 5 meilleures plantes


In [42]:
# Exemple d'utilisation : Entrer un symptôme
symptome_utilisateur = "Je cherche à réduire l'anxiété et favoriser un bon sommeil"
recommandations = recommander_plantes(symptome_utilisateur, df)

# Afficher les recommandations enrichies
print("\nPlantes recommandées :")
for _, row in recommandations.iterrows():
    print(f"\nPlante : {row['Plante']}")
    print(f"Bienfaits : {row['Bienfaits']}")
    print(f"Utilisations : {row['Utilisations']}")
    print(f"Précautions : {row['Précautions']}")
    print(f"Pertinence : {row['score_similarite']:.2f}")


Plantes recommandées :

Plante : Valériane
Bienfaits : Favorise un sommeil réparateur, apaise l'anxiété et réduit le stress
Utilisations : Infusion, teinture
Précautions : Ne pas combiner avec des sédatifs
Pertinence : 0.20

Plante : Lavande
Bienfaits : Apaise l'anxiété, favorise le sommeil, soulage les irritations cutanées
Utilisations : Huile essentielle, infusion
Précautions : À éviter en cas d'allergies respiratoires sévères
Pertinence : 0.19

Plante : Camomille
Bienfaits : Réduit le stress, améliore le sommeil, soulage les crampes abdominales
Utilisations : Infusion, teinture
Précautions : Pas recommandé aux femmes enceintes sans avis médical
Pertinence : 0.09

Plante : Menthe poivrée
Bienfaits : Soulage les maux de tête, les douleurs musculaires et les troubles digestifs
Utilisations : Infusion, huile essentielle
Précautions : Éviter en cas de reflux gastrique ou d'allergie
Pertinence : 0.00

Plante : Aloe Vera
Bienfaits : Apaise les brûlures, hydrate la peau, accélère la cicatr